# Проверить мощность и консервативность критериев Вальда и Фишера для линейной модели (в wald_test поставить флаг use_f=False).


In [133]:
import numpy as np
import random
import statsmodels.api as sm
import scipy.stats as st
import matplotlib.pyplot as plt
import ipywidgets as widg
import seaborn as sns
from tqdm import tqdm_notebook as tqdm


Проверить мощность и консервативность критериев Вальда и Фишера для линейной модели.

Пусть имеется k выборок с $n$ элементами. Линейная модель

In [294]:
sigma = 1
def gen_X(k, n):
    X = np.random.randn(k, n)
    return X

#set to zero weights listed in null
def gen_y(X, null=[]):
    k, n = X.shape
    w = np.random.randn(n)
    w[null] = 0
    y = (np.random.randn(k)*sigma + X.dot(w))
    return y

При генерирации данных, будем случайно выбирать кол-во объектов(200-1000) и признаков(2-100).

Для измерения мощности сгенерируем данные, у которых все признаки значимы(соответсвует альтернативе) и найдем долю результатов теста Вальда и Фишера, когда принимается альтернатива(H1) - это и будет искомая мощность.
 
Для измерения консервативности сгенерируем данные, у которых некое подмножество признаков не значимы(соотвестствующие веса обнулены). После чего найдем ошибку первого рода и проверим консервативность

In [286]:
max = 1000
alpha = 0.05

In [295]:
h1W = 0
h1F = 0
for i in tqdm(range(max)):
    X = gen_X(random.randint(200, 1000),random.randint(2,100))
    res = sm.OLS(gen_y(X), X).fit()
    numOfParams = len(res.params)
    choice = np.random.choice(numOfParams, np.random.randint(1, numOfParams), replace=False)
    R = np.eye(numOfParams)[choice, :]
    if (res.f_test(R).pvalue < alpha):
        h1F += 1
    if (res.wald_test(R, use_f=False).pvalue < alpha):
        h1W += 1         

In [296]:
print("Мощность теста Вальда: {}".format(h1W/max))
print("Мощность теста Фишера: {}".format(h1F/max))

Мощность теста Вальда: 0.997
Мощность теста Фишера: 0.997


Довольно мощно!

In [297]:
h0W = 0
h0F = 0
for i in tqdm(range(max)):
    numOfParams = random.randint(2,100)
    X = gen_X(random.randint(200, 1000), numOfParams)
    choice = np.random.choice(numOfParams, np.random.randint(1, numOfParams), replace=False)
    res = sm.OLS(gen_y(X, choice), X).fit()
    R = np.eye(numOfParams)[choice, :]
    if (res.f_test(R).pvalue < alpha):
        h0F += 1
    if (res.wald_test(R, use_f=False).pvalue < alpha):
        h0W += 1         

In [298]:
print("Ошибка первого рода теста Вальда: {}".format(h0W/max))
print("Ошибка первого рода теста Фишера: {}".format(h0F/max))

Ошибка первого рода теста Вальда: 0.049
Ошибка первого рода теста Фишера: 0.043


A test is conservative if, when constructed for a given nominal significance level, the true probability of incorrectly rejecting the null hypothesis is never greater than the nominal level.

Таким образом, методы на границе консервативности